In [26]:
################ Library imports ################
import numpy as np
import math as math
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import scipy.linalg
import seaborn as sns
import time

import itertools

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel

from sklearn.metrics import mean_squared_error, mean_absolute_error
 
from scipy.optimize import curve_fit

from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

In [51]:
"""
Scenario definition
"""
n_ap = 7

length = 8
width = 19.2

ap_x = 5
ap_y = 5

locations = np.array(np.meshgrid(np.linspace(0, width, ap_x).round(2), np.linspace(0, length, ap_y).round(2))).T.reshape(-1,2)
locations[:, [1, 0]] = locations[:, [0, 1]]

# ----------------------------------------------------------------------
#  AP locations
L_ap = locations[[0,2,4,10,20,22,24],:]

# ----------------------------------------------------------------------
#  VRP locations
L_vrp = np.delete(locations, [0,2,4,10,20,22,24], 0)

In [52]:
"""
Compute Euclidean distance between APs
    
    array --> Array of AP/VRP locations
"""
def compute_euclidean_distance(array):
    n = array.shape[0]

    d_mat = [np.linalg.norm(i - j) for (i,j) in itertools.product(array, array)]
    d_mat = np.array(d_mat).reshape((n, n), order='F')

    return d_mat

In [53]:
d_RTT = compute_euclidean_distance(L_ap)

In [54]:
"""
Gaussian Process Fit using scikit-learn GPR
"""
sigma_n = 0.0

kernel = ConstantKernel(constant_value=100.0,constant_value_bounds=(1e-3, 1e3)) \
            * RBF(length_scale=10.0, length_scale_bounds=(1e-3, 1e3))

gp = GaussianProcessRegressor(kernel=kernel, alpha=sigma_n**2, n_restarts_optimizer=10)

In [55]:
gp.fit(L_ap, d_RTT[3])

GaussianProcessRegressor(alpha=0.0, kernel=10**2 * RBF(length_scale=10),
                         n_restarts_optimizer=10)

In [56]:
pred, sigma = gp.predict(L_vrp, return_cov=True)